In [1]:
# Import the required libraries and dependencies
import pandas as pd
from pathlib import Path

## Import data 

In [2]:
# Import indiegogo dataset
indiegogo_df = pd.read_csv(
    Path('./Resources/indiegogo_data/indiegogo.csv')
)

# Fix ID column
indiegogo_df.rename(columns={'project_id' : 'ID'}, inplace=True)

# Set the index as the ID
indiegogo_df.set_index('ID', inplace=True)

# View head
indiegogo_df.head(2)

,currency,category,year_end,month_end,day_end,time_end,amount_raised,funded_percent,in_demand,year_launch,...,apr,may,jun,jul,aug,sep,oct,nov,dec,tperiod
ID,,,,,,,,,,,,,,,,,,,,,
3936,USD,Transportation,2010,5,12,23:59:00,840,16.80%,False,2010,...,1,0,0,0,0,0,0,0,0,1
5109,USD,Human Rights,2010,7,2,23:59:00,250,20.83%,False,2010,...,0,0,1,0,0,0,0,0,0,2


## Rename cols

In [3]:
# Rename columns to match the kickstarter df
indiegogo_df.rename(columns={'category' : 'main_category', 'title' : 'name', 'amount_raised_usd' : 'usd_pledged_real', 'goal_usd' : 'usd_goal_real'}, inplace=True)

## Clean df

In [4]:
# fix dates
indiegogo_df['launched'] = pd.to_datetime(indiegogo_df['date_launch'] + ' ' + indiegogo_df['time_launch'])
indiegogo_df['deadline'] = pd.to_datetime(indiegogo_df['date_end'] + ' ' + indiegogo_df['time_end'])

## Create dfs from selected columns

In [5]:
# List columns to drop
indiegogo_cols_drop = ['year_end',
'month_end',
'day_end',
'amount_raised',
'in_demand',
'year_launch',
'month_launch',
'day_launch',
'tagline',
'url',
'australia',
'canada',
'switzerland',
'denmark',
'western_europe',
'great_britain',
'hong_kong',
'norway',
'sweden',
'singapore',
'united_states',
'education',
'productivity',
'energy_greentech',
'wellness',
'comics',
'fashion_wearables',
'video_games',
'photography',
'tv_shows',
'dance_theater',
'phones_accessories',
'audio',
'film',
'transportation',
'art',
'environment',
'writing_publishing',
'music',
'travel_outdoors',
'health_fitness',
'tabletop_games',
'home',
'local_business',
'food_beverage',
'culture',
'human_rights',
'podcasts_vlogs',
'camera_gear',
'time_end',
'time_launch',
'date_launch',
'date_end',
'jan',
'feb',
'mar',
'apr',
'may',
'jun',
'jul',
'aug',
'sep',
'oct',
'nov',
'dec',
'tperiod']

# Selected columns indiegogo df
indiegogo_selected_cols_df = indiegogo_df.drop(indiegogo_cols_drop, axis=1)

## Add new cols

In [7]:
# Duration
indiegogo_selected_cols_df['duration'] = indiegogo_selected_cols_df['deadline'] - indiegogo_selected_cols_df['launched'] 
indiegogo_selected_cols_df['duration'] = indiegogo_selected_cols_df['duration'].dt.days

# Daily Goal 
indiegogo_selected_cols_df['daily_goal'] = round(indiegogo_selected_cols_df['usd_goal_real'] / indiegogo_selected_cols_df['duration'],2)

# Daily Pledged
indiegogo_selected_cols_df['daily_pledged'] = round(indiegogo_selected_cols_df['usd_pledged_real'] / indiegogo_selected_cols_df['duration'],2)

# Funded Percentage
indiegogo_selected_cols_df['funded_percent'] = round(indiegogo_selected_cols_df['usd_pledged_real'] / indiegogo_selected_cols_df['usd_goal_real'],4)

# Add country col using currency/country matchings from kickstarter large dataset
# Import kickstarter data
# Import large kickstarter recent dataset
kickstarter_currency_country_cols_df = pd.read_csv(
    Path('./Resources/kickstarter_data/ks-projects-201801.csv'),
    usecols=['currency', 'country']
)

# Imported data has a space at the end of the column name
# Remove spaces in columns name
kickstarter_currency_country_cols_df.columns = kickstarter_currency_country_cols_df.columns.str.replace(' ','')


# Create a dictionary of country and currency pairs to add country col
country_currency_df = kickstarter_currency_country_cols_df.loc[:,['country', 'currency']]
country_currency_df.drop(kickstarter_currency_country_cols_df[kickstarter_currency_country_cols_df['country'] == 'N,0"'].index, inplace=True)
country_currency_df.drop_duplicates(inplace=True)
country_currency_df.set_index('currency', inplace = True)
currency_country_dict = country_currency_df.to_dict()['country']

# Clean up all the N,0" values for countries using the currency_country_dict
indiegogo_selected_cols_df['country'] = indiegogo_selected_cols_df.apply(lambda row: currency_country_dict[row['currency']], axis=1)

# Add full country name col
# Add full country name col
%run ./Util/consts.ipynb
indiegogo_selected_cols_df['full_country_names'] = indiegogo_selected_cols_df['country'].apply(lambda x: full_country_names_dict[x])

# Fix types
indiegogo_selected_cols_df = indiegogo_selected_cols_df.astype({"funded_percent": 'float'})

array(['US', 'LU', 'CA', 'GB', 'AU', 'HK', 'SE', 'CH', 'SG', 'DK', 'NO'],
      dtype=object)

## Reorder cols

In [16]:
# Base column names for reordering
base_order = ['name', 'main_category', 'currency', 'usd_goal_real', 'usd_pledged_real', 'deadline', 'launched', 'state', 'funded_percent','duration', 'daily_goal', 'daily_pledged', 'country', 'full_country_names']
indiegogo_order = base_order

# Reorder each df
indiegogo_selected_cols_df = indiegogo_selected_cols_df[indiegogo_order]

## Write clean data to files

In [17]:
indiegogo_selected_cols_df.to_csv('./Resources/indiegogo_data_clean/indiegogo-projects.csv')